In [23]:
import geopandas as gpd
import pandas as pd
from glob import glob

In [25]:
kdprov=['81']
nmfiledegurba="degurba_Maluku_00.gpkg"

In [26]:
data_degurba_grod=gpd.read_file('data/output/2_grid_degurba/'+nmfiledegurba)

In [27]:
data_pop=gpd.GeoDataFrame()
for i in kdprov:
    gl_pop=glob("data/output/1_population_density/grid_pop_"+i+"*_100m.gpkg")
    for j in gl_pop:
        y_temp=gpd.read_file(j)
        y_temp['geometry']=y_temp.centroid
        y_temp=y_temp[['j_pend','geometry','p_area']]
        data_pop=y_temp.append(y_temp)

/tmp/ipykernel_22879/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_22879/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_22879/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_22879/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_temp.append(y_temp)
/tmp/ipykernel_22879/2622914391.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_pop=y_

In [28]:
data_desa_raster=gpd.GeoDataFrame()
for i in kdprov:
    temp=gpd.read_file('data/output/0_raster_data_desa/'+'desa_pend_rasterize_'+i+'.gpkg')
    data_desa_raster=temp.append(data_desa_raster)

/tmp/ipykernel_22879/2837293180.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data_desa_raster=temp.append(data_desa_raster)


In [29]:
pop_w_grid=data_pop.sjoin_nearest(data_degurba_grod)[['j_pend','classification','geometry']]

In [30]:
data_desa=data_desa_raster.sjoin(pop_w_grid)[['j_pend','classification','iddesa','geometry']]

In [31]:
data_desa

,j_pend,classification,iddesa,geometry
0,0.000000,RURAL,8101040001,"POLYGON ((13072400.000 -977800.000, 13073300.0..."
0,0.000000,RURAL,8101040001,"POLYGON ((13072400.000 -977800.000, 13073300.0..."
0,0.000000,RURAL,8101040001,"POLYGON ((13072400.000 -977800.000, 13073300.0..."
0,0.000000,RURAL,8101040001,"POLYGON ((13072400.000 -977800.000, 13073300.0..."
0,0.000000,RURAL,8101040001,"POLYGON ((13072400.000 -977800.000, 13073300.0..."
...,...,...,...,...
88,12.908012,RURAL,8101055005,"MULTIPOLYGON (((13132100.000 -823700.000, 1313..."
88,0.000000,RURAL,8101055005,"MULTIPOLYGON (((13132100.000 -823700.000, 1313..."
88,0.000000,RURAL,8101055005,"MULTIPOLYGON (((13132100.000 -823700.000, 1313..."
88,0.000000,RURAL,8101055005,"MULTIPOLYGON (((13132100.000 -823700.000, 1313..."


In [32]:
tab_data=pd.pivot_table(data_desa,index='iddesa',columns='classification',
                        values='j_pend',aggfunc='sum').fillna(0).reset_index()

In [33]:
l_col=['CENTRE','CLUSTER','RURAL']
for i in l_col:
    if i not in tab_data.columns:
        tab_data[i]=0
tab_data

classification,iddesa,CLUSTER,RURAL,CENTRE
0,8101040001,0.000000,6176.000000,0
1,8101040002,27156.894216,541.105784,0
2,8101040009,12974.712288,621.287712,0
3,8101040010,6741.349037,2286.650963,0
4,8101040011,0.000000,6020.000000,0
...,...,...,...,...
84,8101055001,0.000000,1222.000000,0
85,8101055002,0.000000,1146.000000,0
86,8101055003,0.000000,1762.000000,0
87,8101055004,0.000000,1626.000000,0


In [34]:
def define_LAU(y):
    ['CENTRE','CLUSTER','RURAL']
    total_pop=y.CENTRE+y.CLUSTER+y.RURAL
    if total_pop==0:
        return 'RURALS'
    elif (y.CENTRE/total_pop)>=0.5:
        return 'CITIES'
    elif (((y.CENTRE/total_pop)<0.5)&((y.RURAL/total_pop)<0.5)):
        return 'TOWNS'
    else:
        return 'RURALS'

In [35]:
tab_data['LAU']=tab_data.apply(lambda y:define_LAU(y),axis=1)

In [36]:
peta_desa=gpd.read_file(r"data/input/Master_Data/DESA_20201_BYINDO.gpkg").query('kdprov in @kdprov')

In [37]:
peta_desa_LAU=peta_desa.merge(tab_data[['iddesa','LAU']],how='left').fillna('RURALS')

In [38]:
for i in kdprov:
    peta_desa_LAU.query("kdprov==@i").to_file(
        'data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+i+'.gpkg',driver='GPKG')

In [ ]:
from dbfread import DBF
mfd_=pd.DataFrame(iter(DBF('data/input/Master_Data/mfd_2020_1.dbf')))

In [ ]:
mfd_['iddesa']=mfd_.PROP+mfd_.KAB+mfd_.KEC+mfd_.DESA

In [ ]:
for i in kdprov:
    peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).query("kdprov==@i").to_file(
        'data/output/3_small_spatial_unit_degurba/small_spatial_unit_'+i+'.gpkg',driver='GPKG')

In [ ]:
peta_desa_LAU.merge(mfd_[['iddesa','DRH']]).pivot_table(index='DRH',columns='LAU',values='iddesa',aggfunc='count').fillna(0)